#이 장에서는 새운 기법인 NMF(Non-negative Matrix Factorization) 을 소개

##데이터 세트에서 독립적인 특성을 찾는데 사용

##독립특성 추출 기법중 하나 

비음수 행렬 V가 주어 졌을때 W * H = V가 성립하는 행렬 W와 H를 구하는 알고리즘

<img src=nmf-01.png>

##어떻게?

### 무작위 행렬 W,H를 최초 생성하고

W * H가 V에 근접할때 까지 반복적으로 W 와 H 를 재구성

###NMF의 특성

W의 각 열이 특성<br>
W의 각 행은 C의 같은 행이 '특성에 얼마나 적합'한지를 표시<br>
즉 특성의 가중치를 의미

H의 각 행이 특성<br>
H의 각 열은 V의 같은 열이 '특성이 얼마나 중요한지를 표시'

<img src=nmf-02.png>

#1. 뉴스 쿠퍼스 수집

In [ ]:
#feedParser 설치 필요

In [ ]:
#feedPartser를 통해 기사를 수집할 RSS Feef 리스트 작성 

feedlist=['http://today.reuters.com/rss/topNews',
          'http://today.reuters.com/rss/domesticNews',
          'http://today.reuters.com/rss/worldNews',
          'http://hosted.ap.org/lineups/TOPHEADS-rss_2.0.xml',
          'http://hosted.ap.org/lineups/USHEADS-rss_2.0.xml',
          'http://hosted.ap.org/lineups/WORLDHEADS-rss_2.0.xml',
          'http://hosted.ap.org/lineups/POLITICSHEADS-rss_2.0.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/HomePage.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/International.xml',
          'http://news.google.com/?output=rss',
          'http://feeds.salon.com/salon/news',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,0,00.rss',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,80,00.rss',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,81,00.rss',
          'http://rss.cnn.com/rss/edition.rss',
          'http://rss.cnn.com/rss/edition_world.rss',
          'http://rss.cnn.com/rss/edition_us.rss']
#필요에 따라 수정 가능. 

In [ ]:
#html CODE 제거 및 단어 분리 
def stripHTML(h):
  p=''
  s=0
  for c in h:
    if c=='<': s=1
    elif c=='>':
      s=0
      p+=' '
    elif s==0: p+=c
  return p


def separatewords(text):
  splitter=re.compile('\\W*')
  return [s.lower() for s in splitter.split(text) if len(s)>3]

In [3]:
# %load newsfeatures.py
import feedparser
import re


feedlist=['http://today.reuters.com/rss/topNews',
          'http://today.reuters.com/rss/domesticNews',
          'http://today.reuters.com/rss/worldNews',
          'http://hosted.ap.org/lineups/TOPHEADS-rss_2.0.xml',
          'http://hosted.ap.org/lineups/USHEADS-rss_2.0.xml',
          'http://hosted.ap.org/lineups/WORLDHEADS-rss_2.0.xml',
          'http://hosted.ap.org/lineups/POLITICSHEADS-rss_2.0.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/HomePage.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/International.xml',
          'http://news.google.com/?output=rss',
          'http://feeds.salon.com/salon/news',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,0,00.rss',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,80,00.rss',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,81,00.rss',
          'http://rss.cnn.com/rss/edition.rss',
          'http://rss.cnn.com/rss/edition_world.rss',
          'http://rss.cnn.com/rss/edition_us.rss']

def stripHTML(h):
  p=''
  s=0
  for c in h:
    if c=='<': s=1
    elif c=='>':
      s=0
      p+=' '
    elif s==0: p+=c
  return p


def separatewords(text):
  splitter=re.compile('\\W*')
  return [s.lower() for s in splitter.split(text) if len(s)>3]

def getarticlewords():
  allwords={}
  articlewords=[]
  articletitles=[]
  ec=0
  # Loop over every feed
  # 모든 피드 마다 루프를 돔
  for feed in feedlist:
    f=feedparser.parse(feed)
    
    # Loop over every article
    for e in f.entries:
      # Ignore identical articles
      # 동일한 기사는 무시
      if e.title in articletitles: continue
      
      # Extract the words
      # 단어를 추출
      txt=e.title.encode('utf8')+stripHTML(e.description.encode('utf8'))
      words=separatewords(txt)
      articlewords.append({})
      articletitles.append(e.title)
      
      # Increase the counts for this word in allwords and in articlewords
      # allowwords, articlewords 에 카운트 증가 
      for word in words:
        allwords.setdefault(word,0)
        allwords[word]+=1
        articlewords[ec].setdefault(word,0)
        articlewords[ec][word]+=1
      ec+=1
  return allwords,articlewords,articletitles

#기사와 단어 행렬을 작성
def makematrix(allw,articlew):
  wordvec=[]
  
  # Only take words that are common but not too common
  for w,c in allw.items():
    if c>3 and c<len(articlew)*0.6:
      wordvec.append(w) 
  
  # Create the word matrix
  l1=[[(word in f and f[word] or 0) for word in wordvec] for f in articlew]
  return l1,wordvec

from numpy import *

def showfeatures(w,h,titles,wordvec,out='features.txt'): 
  outfile=file(out,'w')  
  pc,wc=shape(h)
  toppatterns=[[] for i in range(len(titles))]
  patternnames=[]
  
  # Loop over all the features
  for i in range(pc):
    slist=[]
    # Create a list of words and their weights
    for j in range(wc):
      slist.append((h[i,j],wordvec[j]))
    # Reverse sort the word list
    slist.sort()
    slist.reverse()
    
    # Print the first six elements
    n=[s[1] for s in slist[0:6]]
    outfile.write(str(n)+'\n')
    patternnames.append(n)
    
    # Create a list of articles for this feature
    flist=[]
    for j in range(len(titles)):
      # Add the article with its weight
      flist.append((w[j,i],titles[j]))
      toppatterns[j].append((w[j,i],i,titles[j]))
    
    # Reverse sort the list
    flist.sort()
    flist.reverse()
    
    # Show the top 3 articles
    for f in flist[0:3]:
      outfile.write(str(f)+'\n')
    outfile.write('\n')

  outfile.close()
  # Return the pattern names for later use
  return toppatterns,patternnames

def showarticles(titles,toppatterns,patternnames,out='articles.txt'):
  outfile=file(out,'w')  
  
  # Loop over all the articles
  for j in range(len(titles)):
    outfile.write(titles[j].encode('utf8')+'\n')
    
    # Get the top features for this article and
    # reverse sort them
    toppatterns[j].sort()
    toppatterns[j].reverse()
    
    # Print the top three patterns
    for i in range(3):
      outfile.write(str(toppatterns[j][i][0])+' '+
                    str(patternnames[toppatterns[j][i][1]])+'\n')
    outfile.write('\n')
    
  outfile.close()


In [4]:
import newsfeatures
allw, artw, artt = newsfeatures.getarticlewords()
wordmatrix, wordvec = newsfeatures.makematrix(allw, artw)
wordvec[0:10]

['under',
 'lord',
 'confederate',
 'wednesday',
 'leaders',
 'spokesman',
 'million',
 'never',
 'here',
 'pluto']

In [5]:
wordmatrix[1][0:10]

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

베이지안 분류기를 이용한 분류

In [8]:
def wordmatrixfeatures(x):
    return [wordvec[w] for w in range(len(x)) if x[w] > 0]


In [9]:
wordmatrixfeatures(wordmatrix[0])

['investigation',
 'court',
 'scott',
 'wisconsin',
 'into',
 'walker',
 '2012',
 'ends']

In [11]:
import docclass
classifier = docclass.naivebayes(wordmatrixfeatures)
classifier.setdb('newstest.db')
artt[0]


ImportError: No module named pysqlite2

In [17]:

import sys
reload(sys)
sys.setdefaultencoding("utf-8")

import clusters
clust = clusters.hcluster(wordmatrix)
clusters.drawdendrogram(clust,artt,jpeg='news.jpg')

UnicodeEncodeError: 'latin-1' codec can't encode character u'\u2019' in position 9: ordinal not in range(256)

<img src=news.jpg>